In [ ]:
#| default_exp losses
#| all_polars

# Losses

> Loss functions for model evaluation.
> 

The most important train signal is the forecast error, which is the difference between the observed value $y_{\tau}$ and the prediction $\hat{y}_{\tau}$, at time $y_{\tau}$:

$$
e_{\tau} = y_{\tau}-\hat{y}_{\tau} \qquad \qquad \tau \in \{t+1,\dots,t+H \}
$$

The train loss summarizes the forecast errors in different evaluation metrics.

In [ ]:
#| export
from typing import Callable, Dict, List, Optional, Union

import numpy as np
import pandas as pd

import utilsforecast.processing as ufp
from utilsforecast.compat import DataFrame, pl

In [ ]:
#| hide
import re
import warnings

from nbdev import show_doc

from utilsforecast.compat import POLARS_INSTALLED

In [ ]:
#| hide
warnings.filterwarnings('ignore', message='Unknown section References')

In [ ]:
from utilsforecast.data import generate_series

In [ ]:
models = ['model0', 'model1']
series = generate_series(10, n_models=2, level=[80])

In [ ]:
#| polars
series_pl = generate_series(10, n_models=2, level=[80], engine='polars')

## 1. Scale-dependent Errors

### Mean Absolute Error (MAE)

$$
\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} |y_{\tau} - \hat{y}_{\tau}|
$$

![](imgs/losses/mae_loss.png)

In [ ]:
#| exporti
def _base_docstring(*args, **kwargs) -> Callable:
    base_docstring = """

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, actual values and predictions.
    models : list of str
        Columns that identify the models predictions.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.
    """
    def docstring_decorator(f: Callable):
        if f.__doc__ is not None:
            f.__doc__ += base_docstring
        return f

    return docstring_decorator(*args, **kwargs)

In [ ]:
#| export
class NotSet: ...

NOT_SET = NotSet()
AGG_BY = Optional[Union[str, List[str], List[Optional[List[str]]], NotSet]]

In [ ]:
#| exporti
def _aggregate(
    df: DataFrame, models: List[str], id_col: str, agg_by: AGG_BY
) -> Union[DataFrame, pd.Series]:
    aggs = {m: 'mean' for m in models}
    if agg_by == NOT_SET:
        res = ufp.group_by_agg(df, id_col, aggs, maintain_order=True)
    elif (
        isinstance(agg_by, str)
        or (isinstance(agg_by, list) and isinstance(agg_by[0], str))
    ):
        res = ufp.group_by_agg(df, agg_by, aggs, maintain_order=True)
    elif isinstance(agg_by, list):
        res = df
        for by in agg_by:
            res = _aggregate(res, models, id_col, by)
    elif agg_by is None:
        res = df[models].mean()
        if isinstance(res, pd.Series):
            res = res.to_frame().T
    else:
        raise ValueError(
            "`agg_by` must be str, list of str, list of list of str or `None`. "
            f"got: {type(agg_by)}"
        )
    return res

In [ ]:
#| export
@_base_docstring
def mae(
    df: DataFrame,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> Union[DataFrame, pd.Series]:
    """Mean Absolute Error (MAE)

    MAE measures the relative prediction
    accuracy of a forecasting method by calculating the
    deviation of the prediction and the true
    value at a given time and averages these devations
    over the length of the series."""
    df = ufp.copy_if_pandas(df, deep=False)
    if isinstance(df, pd.DataFrame):
        df[models] = df[models].sub(df[target_col], axis=0).abs()
    else:
        def gen_expr(model):
            return pl.col(target_col).sub(pl.col(model)).abs().alias(model)

        df = df.with_columns(*[gen_expr(m) for m in models])
    return _aggregate(df=df, models=models, id_col=id_col, agg_by=agg_by)

In [ ]:
series2 = pd.concat([series.assign(w=0), series.assign(w=1)])

In [ ]:
mae(series, models)

,unique_id,model0,model1
0,0,0.158108,0.163246
1,1,0.160109,0.143805
2,2,0.159815,0.170510
3,3,0.168537,0.161595
4,4,0.170182,0.163329
5,5,0.165729,0.165845
6,6,0.166688,0.155721
7,7,0.154513,0.162503
8,8,0.152352,0.173123
9,9,0.156832,0.163132


In [ ]:
mae(series2, models, agg_by=[['w', 'unique_id'], None])

,model0,model1
0,0.161286,0.162281


In [ ]:
mae(series2, models, agg_by=['w', 'unique_id'])[models].mean().to_frame().T

,model0,model1
0,0.161286,0.162281


In [ ]:
mae(series, models, agg_by=[['unique_id'], None])

,model0,model1
0,0.161286,0.162281


In [ ]:
mae(series, models)[models].mean().to_frame().T

,model0,model1
0,0.161286,0.162281


In [ ]:
mae(series, models, agg_by=None)

,model0,model1
0,0.162216,0.162466


In [ ]:
mae(series_pl, models, agg_by=None)

model0,model1
f64,f64
0.162216,0.162466


In [ ]:
show_doc(mae, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L80){target="_blank" style="float:right; font-size:smaller"}

#### mae

>      mae
>           (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFra
>           me], models:List[str], id_col:str='unique_id', target_col:str='y', a
>           gg_by:Union[str,List[str],List[Optional[List[str]]],__main__.NotSet,
>           NoneType]=<__main__.NotSet object>)

*Mean Absolute Error (MAE)

MAE measures the relative prediction
accuracy of a forecasting method by calculating the
deviation of the prediction and the true
value at a given time and averages these devations
over the length of the series.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
def pd_vs_pl(pd_df, pl_df, models):
    np.testing.assert_allclose(
        pd_df[models].to_numpy(),
        pl_df.sort('unique_id').select(models).to_numpy(),
    )

In [ ]:
#| polars
pd_vs_pl(
    mae(series, models),
    mae(series_pl, models),
    models,
)

### Mean Squared Error

$$
\mathrm{MSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} (y_{\tau} - \hat{y}_{\tau})^{2}
$$

![](imgs/losses/mse_loss.png)

In [ ]:
#| export
@_base_docstring
def mse(
    df: DataFrame,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> Union[DataFrame, pd.Series]:
    """Mean Squared Error (MSE)
    
    MSE measures the relative prediction
    accuracy of a forecasting method by calculating the 
    squared deviation of the prediction and the true
    value at a given time, and averages these devations
    over the length of the series."""
    df = ufp.copy_if_pandas(df, deep=False)
    if isinstance(df, pd.DataFrame):
        df[models] = df[models].sub(df[target_col], axis=0).pow(2)
    else:
        def gen_expr(model):
            return pl.col(target_col).sub(pl.col(model)).pow(2).alias(model)

        df = df.with_columns(*[gen_expr(m) for m in models])
    return _aggregate(df=df, models=models, id_col=id_col, agg_by=agg_by)

In [ ]:
show_doc(mse, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L107){target="_blank" style="float:right; font-size:smaller"}

#### mse

>      mse
>           (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFra
>           me], models:List[str], id_col:str='unique_id', target_col:str='y', a
>           gg_by:Union[str,List[str],List[Optional[List[str]]],__main__.NotSet,
>           NoneType]=<__main__.NotSet object>)

*Mean Squared Error (MSE)

MSE measures the relative prediction
accuracy of a forecasting method by calculating the 
squared deviation of the prediction and the true
value at a given time, and averages these devations
over the length of the series.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
mse(series, models, agg_by=[['unique_id'], None])

,model0,model1
0,0.048653,0.049198


In [ ]:
#| polars
pd_vs_pl(
    mse(series, models),
    mse(series_pl, models),
    models,
)

### Root Mean Squared Error

$$
\mathrm{RMSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \sqrt{\frac{1}{H} \sum^{t+H}_{\tau=t+1} (y_{\tau} - \hat{y}_{\tau})^{2}}
$$

![](imgs/losses/rmse_loss.png)

In [ ]:
#| export
@_base_docstring
def rmse(
    df: DataFrame,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> Union[DataFrame, pd.Series]:
    """Root Mean Squared Error (RMSE)
    
    RMSE measures the relative prediction
    accuracy of a forecasting method by calculating the squared deviation
    of the prediction and the observed value at a given time and
    averages these devations over the length of the series.
    Finally the RMSE will be in the same scale
    as the original time series so its comparison with other
    series is possible only if they share a common scale. 
    RMSE has a direct connection to the L2 norm."""
    if agg_by == NOT_SET or agg_by is None or (isinstance(agg_by, list) and not isinstance(agg_by[0], list)):
        res = mse(df, models, id_col, target_col, agg_by)
        remaining_aggs = []
    else:
        res = mse(df, models, id_col, target_col, agg_by[0])
        remaining_aggs = agg_by[1:]
    if isinstance(res, (pd.DataFrame, pd.Series)):
        res[models] = res[models].pow(0.5)
    else:
        res = res.with_columns(*[pl.col(c).pow(0.5) for c in models])
    if remaining_aggs:
        res = _aggregate(res, models, id_col, remaining_aggs)
    return res

In [ ]:
rmse(series, models)

,unique_id,model0,model1
0,0,0.215680,0.221793
1,1,0.228856,0.199390
2,2,0.213222,0.236165
3,3,0.227837,0.217415
4,4,0.234706,0.220761
5,5,0.222982,0.221940
6,6,0.229617,0.215380
7,7,0.211507,0.220508
8,8,0.200869,0.241552
9,9,0.218295,0.220528


In [ ]:
rmse(series, models, agg_by=[['unique_id'], None])

,model0,model1
0,0.220357,0.221543


In [ ]:
rmse(series, models)[models].mean()

model0    0.220357
model1    0.221543
dtype: float64

In [ ]:
rmse(series_pl, models)[models].mean()

model0,model1
f64,f64
0.220357,0.221543


In [ ]:
show_doc(rmse, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L134){target="_blank" style="float:right; font-size:smaller"}

#### rmse

>      rmse
>            (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFr
>            ame], models:List[str], id_col:str='unique_id', target_col:str='y',
>            agg_by:Union[str,List[str],List[Optional[List[str]]],__main__.NotSe
>            t,NoneType]=<__main__.NotSet object>)

*Root Mean Squared Error (RMSE)

RMSE measures the relative prediction
accuracy of a forecasting method by calculating the squared deviation
of the prediction and the observed value at a given time and
averages these devations over the length of the series.
Finally the RMSE will be in the same scale
as the original time series so its comparison with other
series is possible only if they share a common scale. 
RMSE has a direct connection to the L2 norm.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    rmse(series, models),
    rmse(series_pl, models),
    models,
)

## 2. Percentage Errors

### Mean Absolute Percentage Error

$$
\mathrm{MAPE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{|y_{\tau}|}
$$

![](imgs/losses/mape_loss.png)

In [ ]:
#| exporti
def _zero_to_nan(series: Union[pd.Series, 'pl.Expr']) -> Union[pd.Series, 'pl.Expr']:
    if isinstance(series, pd.Series):
        res = series.replace(0, np.nan)
    else:
        res = (
            pl.when(series == 0)
            .then(float('nan'))
            .otherwise(series.abs())
        )
    return res

In [ ]:
#| export
@_base_docstring
def mape(
    df: DataFrame,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> Union[DataFrame, pd.Series]:
    """Mean Absolute Percentage Error (MAPE)
    
    MAPE measures the relative prediction
    accuracy of a forecasting method by calculating the percentual deviation
    of the prediction and the observed value at a given time and
    averages these devations over the length of the series.
    The closer to zero an observed value is, the higher penalty MAPE loss
    assigns to the corresponding error."""
    df = ufp.copy_if_pandas(df, deep=False)
    if isinstance(df, pd.DataFrame):
        df[models] = (
            df[models]
            .sub(df[target_col], axis=0)
            .abs()
            .div(_zero_to_nan(df[target_col].abs()), axis=0)
        )
    else:
        def gen_expr(model):
            abs_err = pl.col(target_col).sub(pl.col(model)).abs()
            abs_target = _zero_to_nan(pl.col(target_col))
            ratio = abs_err.truediv(abs_target).alias(model)
            return ratio.fill_nan(None)

        df = df.with_columns(*[gen_expr(m) for m in models])
    return _aggregate(df=df, models=models, id_col=id_col, agg_by=agg_by)

In [ ]:
show_doc(mape, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L179){target="_blank" style="float:right; font-size:smaller"}

#### mape

>      mape
>            (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFr
>            ame], models:List[str], id_col:str='unique_id', target_col:str='y',
>            agg_by:Union[str,List[str],List[Optional[List[str]]],__main__.NotSe
>            t,NoneType]=<__main__.NotSet object>)

*Mean Absolute Percentage Error (MAPE)

MAPE measures the relative prediction
accuracy of a forecasting method by calculating the percentual deviation
of the prediction and the observed value at a given time and
averages these devations over the length of the series.
The closer to zero an observed value is, the higher penalty MAPE loss
assigns to the corresponding error.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    mape(series, models),
    mape(series_pl, models),
    models,
)

### Symmetric Mean Absolute Percentage Error

$$
\mathrm{SMAPE}_{2}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{|y_{\tau}|+|\hat{y}_{\tau}|}
$$

In [ ]:
#| export
@_base_docstring
def smape(
    df: DataFrame,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> Union[DataFrame, pd.Series]:
    """Symmetric Mean Absolute Percentage Error (SMAPE)

    SMAPE measures the relative prediction
    accuracy of a forecasting method by calculating the relative deviation
    of the prediction and the observed value scaled by the sum of the
    absolute values for the prediction and observed value at a
    given time, then averages these devations over the length
    of the series. This allows the SMAPE to have bounds between
    0% and 100% which is desirable compared to normal MAPE that
    may be undetermined when the target is zero."""
    df = ufp.copy_if_pandas(df, deep=False)
    if isinstance(df, pd.DataFrame):
        delta_y = df[models].sub(df[target_col], axis=0).abs()
        scale = df[models].abs().add(df[target_col].abs(), axis=0)
        df[models] = delta_y.div(scale).fillna(0)
    else:
        def gen_expr(model):
            abs_err = pl.col(model).sub(pl.col(target_col)).abs()
            denominator = _zero_to_nan(pl.col(model).abs().add(pl.col(target_col)).abs())
            ratio = abs_err.truediv(denominator).alias(model)
            return ratio.fill_nan(0)

        df = df.with_columns(*[gen_expr(m) for m in models])
    return _aggregate(df=df, models=models, id_col=id_col, agg_by=agg_by)

In [ ]:
show_doc(smape, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L215){target="_blank" style="float:right; font-size:smaller"}

#### smape

>      smape
>             (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataF
>             rame], models:List[str], id_col:str='unique_id',
>             target_col:str='y', agg_by:Union[str,List[str],List[Optional[List[
>             str]]],__main__.NotSet,NoneType]=<__main__.NotSet object at
>             0x7f0c605e09a0>)

*Symmetric Mean Absolute Percentage Error (SMAPE)

SMAPE measures the relative prediction
accuracy of a forecasting method by calculating the relative deviation
of the prediction and the observed value scaled by the sum of the
absolute values for the prediction and observed value at a
given time, then averages these devations over the length
of the series. This allows the SMAPE to have bounds between
0% and 100% which is desirable compared to normal MAPE that
may be undetermined when the target is zero.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    smape(series, models),
    smape(series_pl, models),
    models,
)

## 3. Scale-independent Errors

### Mean Absolute Scaled Error

$$
\mathrm{MASE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau}) = 
\frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau})}
$$

![](imgs/losses/mase_loss.png)

In [ ]:
#| export
def mase(
    df: DataFrame,
    models: List[str],
    seasonality: int,
    train_df: DataFrame,
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> Union[DataFrame, pd.Series]:
    """Mean Absolute Scaled Error (MASE)
    
    MASE measures the relative prediction
    accuracy of a forecasting method by comparinng the mean absolute errors
    of the prediction and the observed value against the mean
    absolute errors of the seasonal naive model.
    The MASE partially composed the Overall Weighted Average (OWA), 
    used in the M4 Competition.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, actuals and predictions.
    models : list of str
        Columns that identify the models predictions.
    seasonality : int
        Main frequency of the time series;
        Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1.
    train_df : pandas or polars DataFrame
        Training dataframe with id and actual values. Must be sorted by time.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://robjhyndman.com/papers/mase.pdf        
    """
    mean_abs_err = mae(df, models, id_col, target_col, agg_by)
    if isinstance(train_df, pd.DataFrame):
        # assume train_df is sorted
        lagged = train_df.groupby(id_col, observed=True)[target_col].shift(seasonality)
        scale = ufp.copy_if_pandas(train_df, deep=False)
        scale['_scale'] = scale[target_col].sub(lagged).abs()
    else:
        # assume train_df is sorted
        lagged = pl.col(target_col).shift(seasonality).over(id_col)
        scale = train_df.with_columns(
            pl.col(target_col).sub(lagged).abs().alias('_scale')
        )
    scale = _aggregate(scale, models=['_scale'], id_col=id_col, agg_by=agg_by)
    join_cols = [c for c in mean_abs_err.columns if c not in models]
    if join_cols:
        res = ufp.join(mean_abs_err, scale, on=join_cols)
    else:
        res = ufp.horizontal_concat([mean_abs_err, scale])
    if isinstance(res, pd.DataFrame):
        res[models] = res[models].div(res['_scale'], axis=0)
    else:
        res = res.with_columns(*[pl.col(m) / pl.col('_scale') for m in models])
    return ufp.drop_columns(res, ['_scale'])

In [ ]:
mase(series, models, seasonality=7, train_df=series)

,unique_id,model0,model1
0,0,0.985713,1.017747
1,1,0.923322,0.829304
2,2,0.932431,0.994832
3,3,1.053972,1.010562
4,4,0.977438,0.938081
5,5,0.965258,0.965930
6,6,0.934797,0.873294
7,7,0.939488,0.988070
8,8,1.051247,1.194574
9,9,0.904797,0.941142


In [ ]:
mase(series, models, seasonality=7, train_df=series, agg_by=None)

,model0,model1
0,0.960597,0.962081


In [ ]:
mase(series, models, seasonality=7, train_df=series, agg_by=[['unique_id'], None])

,model0,model1
0,0.964661,0.97061


In [ ]:
show_doc(mase, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L251){target="_blank" style="float:right; font-size:smaller"}

#### mase

>      mase
>            (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFr
>            ame], models:List[str], seasonality:int, train_df:Union[pandas.core
>            .frame.DataFrame,polars.dataframe.frame.DataFrame],
>            id_col:str='unique_id', target_col:str='y', agg_by:Union[str,List[s
>            tr],List[Optional[List[str]]],__main__.NotSet,NoneType]=<__main__.N
>            otSet object at 0x7f0c605e09a0>)

*Mean Absolute Scaled Error (MASE)

MASE measures the relative prediction
accuracy of a forecasting method by comparinng the mean absolute errors
of the prediction and the observed value against the mean
absolute errors of the seasonal naive model.
The MASE partially composed the Overall Weighted Average (OWA), 
used in the M4 Competition.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, actuals and predictions. |
| models | List |  | Columns that identify the models predictions. |
| seasonality | int |  | Main frequency of the time series;<br>Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1. |
| train_df | Union |  | Training dataframe with id and actual values. Must be sorted by time. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    mase(series, models, 7, series),
    mase(series_pl, models, 7, series_pl),
    models,
)

### Relative Mean Absolute Error

$$
\mathrm{RMAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{base}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{base}_{\tau})}
$$

![](imgs/losses/rmae_loss.png)

In [ ]:
#| export
def rmae(
    df: DataFrame,
    models: List[str],
    baseline_models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> DataFrame:
    """Relative Mean Absolute Error (RMAE)
    
    Calculates the RAME between two sets of forecasts (from two different forecasting methods).
    A number smaller than one implies that the forecast in the 
    numerator is better than the forecast in the denominator.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : list of str
        Columns that identify the models predictions.
    baseline_models : list of str
        Columns that identify the baseline models predictions.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.
    """
    numerator = mae(df, models, id_col, target_col, agg_by)
    denominator = mae(df, baseline_models, id_col, target_col, agg_by)
    denom_rename = {m: f'{m}_denominator' for m in models}
    denominator = ufp.rename(denominator, denom_rename)
    join_cols = [c for c in numerator.columns if c not in models]
    if join_cols:
        res = ufp.join(numerator, denominator, on=join_cols)
    else:
        res = ufp.horizontal_concat([numerator, denominator])
    if isinstance(numerator, pd.DataFrame):
        for model, baseline in zip(models, baseline_models):
            col_name = f'{model}_div_{baseline}'
            res[col_name] = res[model].div(_zero_to_nan(res[f'{baseline}_denominator'])).fillna(0)
    else:
        def gen_expr(model, baseline):
            denominator = _zero_to_nan(pl.col(f'{baseline}_denominator'))
            return pl.col(model).truediv(denominator).fill_nan(0).alias(f'{model}_div_{baseline}')

        exprs = [gen_expr(m1, m2) for m1, m2 in zip(models, baseline_models)]
        res = res.with_columns(*exprs)
    model_cols = [
        f'{model}_div_{baseline}' for model, baseline in zip(models, baseline_models)
    ]
    return res[join_cols + model_cols]

In [ ]:
show_doc(rmae, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L319){target="_blank" style="float:right; font-size:smaller"}

#### rmae

>      rmae
>            (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.DataFr
>            ame], models:List[str], baseline_models:List[str],
>            id_col:str='unique_id', target_col:str='y', agg_by:Union[str,List[s
>            tr],List[Optional[List[str]]],__main__.NotSet,NoneType]=<__main__.N
>            otSet object at 0x7f0c605e09a0>)

*Relative Mean Absolute Error (RMAE)

Calculates the RAME between two sets of forecasts (from two different forecasting methods).
A number smaller than one implies that the forecast in the 
numerator is better than the forecast in the denominator.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| models | List |  | Columns that identify the models predictions. |
| baseline_models | List |  | Columns that identify the baseline models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    rmae(series, models, list(reversed(models))),
    rmae(series_pl, models, list(reversed(models))),
    [f'{m1}_div_{m2}' for m1, m2 in zip(models, reversed(models))],
)

## 4. Probabilistic Errors

### Quantile Loss

$$
\mathrm{QL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q)}_{\tau}) = 
\frac{1}{H} \sum^{t+H}_{\tau=t+1} 
\Big( (1-q)\,( \hat{y}^{(q)}_{\tau} - y_{\tau} )_{+} 
+ q\,( y_{\tau} - \hat{y}^{(q)}_{\tau} )_{+} \Big)
$$

![](imgs/losses/q_loss.png)

In [ ]:
#| export
def quantile_loss(
    df: DataFrame,
    models: Dict[str, str],
    q: float = 0.5,
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> DataFrame:
    """Quantile Loss (QL)

    QL measures the deviation of a quantile forecast.
    By weighting the absolute deviation in a non symmetric way, the
    loss pays more attention to under or over estimation.    
    A common value for q is 0.5 for the deviation from the median.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to str
        Mapping from model name to the model predictions for the specified quantile.
    q : float (default=0.5)
        Quantile for the predictions' comparison.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.
    """
    model_names = list(models.keys())
    model_preds = list(models.values())
    df = ufp.copy_if_pandas(df, deep=False)
    if isinstance(df, pd.DataFrame):
        df[model_preds] = -df[model_preds].sub(df[target_col], axis=0)
        df[model_names] = np.maximum(q * df[model_preds], (q - 1) * df[model_preds])
    else:
        def gen_expr(model_name, pred_col):
            delta_y = pl.col(target_col).sub(pl.col(pred_col))
            try:
                col_max = pl.max_horizontal([q * delta_y, (q - 1) * delta_y])
            except AttributeError:
                col_max = pl.max([q * delta_y, (q - 1) * delta_y])
            return col_max.alias(model_name)

        df = df.with_columns(*[gen_expr(name, pred) for name, pred in models.items()])
    return _aggregate(df, model_names, id_col, agg_by)

In [ ]:
show_doc(quantile_loss, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L379){target="_blank" style="float:right; font-size:smaller"}

#### quantile_loss

>      quantile_loss
>                     (df:Union[pandas.core.frame.DataFrame,polars.dataframe.fra
>                     me.DataFrame], models:Dict[str,str], q:float=0.5,
>                     id_col:str='unique_id', target_col:str='y', agg_by:Union[s
>                     tr,List[str],List[Optional[List[str]]],__main__.NotSet,Non
>                     eType]=<__main__.NotSet object>)

*Quantile Loss (QL)

QL measures the deviation of a quantile forecast.
By weighting the absolute deviation in a non symmetric way, the
loss pays more attention to under or over estimation.    
A common value for q is 0.5 for the deviation from the median.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions for the specified quantile. |
| q | float | 0.5 | Quantile for the predictions' comparison. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| hide
df = pd.DataFrame({
    "unique_id": [0, 1, 2],
    "y": [1.0, 2.0, 3.0],
    "overestimation": [2.0, 3.0, 4.0], # y + 1.
    "underestimation": [0.0, 1.0, 2.0], # y - 1.
})
df["unique_id"] = df["unique_id"].astype("category")
df = pd.concat([df, df.assign(unique_id=2)]).reset_index(drop=True)

ql_models_test = ["overestimation", "underestimation"]
quantiles = np.array([0.1, 0.9])

for q in quantiles:
    ql_df = quantile_loss(df, models=dict(zip(ql_models_test, ql_models_test)), q=q)
    # for overestimation, delta_y = y - y_hat = -1 so ql = max(-q, -(q-1)) 
    assert all(max(-q, -(q - 1)) == ql for ql in ql_df["overestimation"])
    # for underestimation, delta_y = y - y_hat = 1, so ql = max(q, q-1)
    assert all(max(q, q - 1) == ql for ql in ql_df["underestimation"])

In [ ]:
#| hide
#| polars
q_models = {
    0.1: {
        'model0': 'model0-lo-80',
        'model1': 'model1-lo-80',
    },
    0.9: {
        'model0': 'model0-hi-80',
        'model1': 'model1-hi-80',
    },
}

for q in quantiles:
    pd_vs_pl(
        quantile_loss(series, q_models[q], q=q),
        quantile_loss(series_pl, q_models[q], q=q),
        models,
    )

### Multi-Quantile Loss

$$
\mathrm{MQL}(\mathbf{y}_{\tau},
[\mathbf{\hat{y}}^{(q_{1})}_{\tau}, ... ,\hat{y}^{(q_{n})}_{\tau}]) = 
\frac{1}{n} \sum_{q_{i}} \mathrm{QL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q_{i})}_{\tau})
$$

![](imgs/losses/mq_loss.png)

In [ ]:
#| export
def mqloss(
    df: DataFrame,
    models: Dict[str, List[str]],
    quantiles: np.ndarray,
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> DataFrame:
    """Multi-Quantile loss (MQL)
    
    MQL calculates the average multi-quantile Loss for
    a given set of quantiles, based on the absolute 
    difference between predicted quantiles and observed values.

    The limit behavior of MQL allows to measure the accuracy 
    of a full predictive distribution $\mathbf{\hat{F}}_{\\tau}$ with 
    the continuous ranked probability score (CRPS). This can be achieved 
    through a numerical integration technique, that discretizes the quantiles 
    and treats the CRPS integral with a left Riemann approximation, averaging over 
    uniformly distanced quantiles.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to list of str
        Mapping from model name to the model predictions for each quantile.
    quantiles : numpy array
        Quantiles to compare against.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://www.jstor.org/stable/2629907
    """
    error = np.empty((df.shape[0], quantiles.size))
    df = ufp.copy_if_pandas(df)
    for model, predictions in models.items():
        for j, q_preds in enumerate(predictions):
            error[:, j] = (df[target_col] - df[q_preds]).to_numpy()
        loss = np.maximum(error * quantiles, error * (quantiles - 1)).mean(axis=1)
        df = ufp.assign_columns(df, model, loss)
    return _aggregate(df, list(models.keys()), id_col, agg_by)

In [ ]:
show_doc(mqloss, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L441){target="_blank" style="float:right; font-size:smaller"}

#### mqloss

>      mqloss
>              (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.Data
>              Frame], models:Dict[str,List[str]], quantiles:numpy.ndarray,
>              id_col:str='unique_id', target_col:str='y', agg_by:Union[str,List
>              [str],List[Optional[List[str]]],__main__.NotSet,NoneType]=<__main
>              __.NotSet object at 0x7f0c605e09a0>)

*Multi-Quantile loss (MQL)

MQL calculates the average multi-quantile Loss for
a given set of quantiles, based on the absolute 
difference between predicted quantiles and observed values.

The limit behavior of MQL allows to measure the accuracy 
of a full predictive distribution $\mathbf{\hat{F}}_{\tau}$ with 
the continuous ranked probability score (CRPS). This can be achieved 
through a numerical integration technique, that discretizes the quantiles 
and treats the CRPS integral with a left Riemann approximation, averaging over 
uniformly distanced quantiles.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions for each quantile. |
| quantiles | ndarray |  | Quantiles to compare against. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| hide
#| polars
mq_models = {
    'model0': ['model0-lo-80', 'model0-hi-80'],
    'model1': ['model1-lo-80', 'model1-hi-80'],
}

expected = pd.concat(
    [
        quantile_loss(series, models=q_models[q], q=q)
        for i, q in enumerate(quantiles)
    ]
).groupby('unique_id', observed=True, as_index=False).mean()
actual = mqloss(
    series,
    models=mq_models,
    quantiles=quantiles,
)
pd.testing.assert_frame_equal(actual, expected)

In [ ]:
#| polars
pd_vs_pl(
    mqloss(series, mq_models, quantiles=quantiles),
    mqloss(series_pl, mq_models, quantiles=quantiles),
    models,
)

In [ ]:
#| hide
#| polars
for series_df in [series, series_pl]:
    if isinstance(series_df, pd.DataFrame):
        df_test = series_df.assign(unique_id=lambda df: df["unique_id"].astype(str))
    else:
        df_test = series_df.with_columns(pl.col("unique_id").cast(pl.Utf8))
    mql_df = mqloss(
        df_test,
        mq_models, 
        quantiles=quantiles,
    )
    assert mql_df.shape == (series["unique_id"].nunique(), 1 + len(models))
    if isinstance(mql_df, pd.DataFrame):
        null_vals = mql_df.isna().sum().sum()
    else:
        null_vals = series_df.select(pl.all().is_null().sum()).sum_horizontal()
    assert null_vals.item() == 0

### Coverage

In [ ]:
#| export
def coverage(
    df: DataFrame,
    models: List[str],
    level: int,
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> DataFrame:
    """Coverage of y with y_hat_lo and y_hat_hi.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : list of str
        Columns that identify the models predictions.
    level : int
        Confidence level used for intervals.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://www.jstor.org/stable/2629907        
    """
    df = ufp.copy_if_pandas(df, deep=False)
    if isinstance(df, pd.DataFrame):
        for model in models:
            df[model] = df[target_col].between(df[f'{model}-lo-{level}'], df[f'{model}-hi-{level}'])
    else:
        def gen_expr(model):
            return pl.col(target_col).is_between(pl.col(f'{model}-lo-{level}'), pl.col(f'{model}-hi-{level}')).alias(model)

        df = df.with_columns(*[gen_expr(m) for m in models])
    return _aggregate(df, models, id_col, agg_by)

In [ ]:
show_doc(coverage, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L500){target="_blank" style="float:right; font-size:smaller"}

#### coverage

>      coverage
>                (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.Da
>                taFrame], models:List[str], level:int, id_col:str='unique_id',
>                target_col:str='y', agg_by:Union[str,List[str],List[Optional[Li
>                st[str]]],__main__.NotSet,NoneType]=<__main__.NotSet object at
>                0x7f0c605e09a0>)

*Coverage of y with y_hat_lo and y_hat_hi.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| models | List |  | Columns that identify the models predictions. |
| level | int |  | Confidence level used for intervals. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    coverage(series, models, 80),
    coverage(series_pl, models, 80),
    models,
)

### Calibration

In [ ]:
#| export
def calibration(
    df: DataFrame,
    models: Dict[str, str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> DataFrame:
    """
    Fraction of y that is lower than the model's predictions. 
    
    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to str
        Mapping from model name to the model predictions.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.
        
    References
    ----------
    [1] https://www.jstor.org/stable/2629907            
    """
    df = ufp.copy_if_pandas(df, deep=False)
    if isinstance(df, pd.DataFrame):
        for name, preds in models.items():
            df[name] = df[target_col].le(df[preds])
    else:
        def gen_expr(model_name, q_preds):
            return pl.col(target_col).le(pl.col(q_preds)).alias(model_name)

        df = df.with_columns(*[gen_expr(name, pred) for name, pred in models.items()])
    return _aggregate(df, list(models.keys()), id_col, agg_by)    

In [ ]:
show_doc(calibration, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L559){target="_blank" style="float:right; font-size:smaller"}

#### calibration

>      calibration
>                   (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame
>                   .DataFrame], models:Dict[str,str], id_col:str='unique_id',
>                   target_col:str='y', agg_by:Union[str,List[str],List[Optional
>                   [List[str]]],__main__.NotSet,NoneType]=<__main__.NotSet
>                   object at 0x7f0c605e09a0>)

*Fraction of y that is lower than the model's predictions.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    calibration(series, q_models[0.9]),
    calibration(series_pl, q_models[0.9]),
    models,
)

### CRPS

$$
\mathrm{sCRPS}(\hat{F}_{\tau}, \mathbf{y}_{\tau}) = \frac{2}{N} \sum_{i}
\int^{1}_{0} \frac{\mathrm{QL}(\hat{F}_{i,\tau}, y_{i,\tau})_{q}}{\sum_{i} | y_{i,\tau} |} dq
$$

Where $\hat{F}_{\tau}$ is the an estimated multivariate distribution, and $y_{i,\tau}$ are its realizations. 

In [ ]:
#| export
def scaled_crps(
    df: DataFrame,
    models: Dict[str, List[str]],
    quantiles: np.ndarray,
    id_col: str = 'unique_id',
    target_col: str = 'y',
    agg_by: AGG_BY = NOT_SET,
) -> DataFrame:
    """Scaled Continues Ranked Probability Score
    
    Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
    to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.
    This metric averages percentual weighted absolute deviations as 
    defined by the quantile losses.
    
    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to list of str
        Mapping from model name to the model predictions for each quantile.
    quantiles : numpy array
        Quantiles to compare against.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://proceedings.mlr.press/v139/rangapuram21a.html        
    """
    eps = np.finfo(float).eps
    quantiles = np.asarray(quantiles)
    loss = mqloss(df, models, quantiles, id_col, target_col, NOT_SET)
    sizes = ufp.counts_by_id(df, id_col)
    if isinstance(loss, pd.DataFrame):
        loss = loss.set_index(id_col)
        sizes = sizes.set_index(id_col)
        assert isinstance(df, pd.DataFrame)
        norm = df[target_col].abs().groupby(df[id_col], observed=True).sum()
        res = 2 * loss.mul(sizes['counts'], axis=0).div(norm + eps, axis=0)
    else:
        def gen_expr(model):
            return (2 * pl.col(model) * pl.col('counts') / (pl.col('norm') + eps)).alias(model)

        grouped_df = ufp.group_by(df, id_col)
        norm = grouped_df.agg(pl.col(target_col).abs().sum().alias('norm'))
        exprs = [gen_expr(m) for m in models.keys()]
        res = loss.join(sizes, on=id_col).join(norm, on=id_col)
        res = res.select([id_col, *exprs])
        res = ufp.group_by(res, id_col, maintain_order=True).mean()
    return _aggregate(res, list(models.keys()), id_col, agg_by)

In [ ]:
show_doc(scaled_crps, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L609){target="_blank" style="float:right; font-size:smaller"}

#### scaled_crps

>      scaled_crps
>                   (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame
>                   .DataFrame], models:Dict[str,List[str]],
>                   quantiles:numpy.ndarray, id_col:str='unique_id',
>                   target_col:str='y', agg_by:Union[str,List[str],List[Optional
>                   [List[str]]],__main__.NotSet,NoneType]=<__main__.NotSet
>                   object at 0x7f0c605e09a0>)

*Scaled Continues Ranked Probability Score

Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.
This metric averages percentual weighted absolute deviations as 
defined by the quantile losses.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions for each quantile. |
| quantiles | ndarray |  | Quantiles to compare against. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| agg_by | Union | <__main__.NotSet object> |  |
| **Returns** | **Union** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    scaled_crps(series, mq_models, quantiles),
    scaled_crps(series_pl, mq_models, quantiles),
    models,
)